In [ ]:
import argparse
import os
import sys
from os import mkdir

import numpy as np
import torch
import torch.nn.functional as F

sys.path.append('..')

from data import make_data_loader
from engine.trainer import do_train
from modeling import build_model
from solver import make_optimizer, WarmupMultiStepLR
from layers import make_loss
from config import cfg
from utils.logger import setup_logger
from utils.feats_pca import feats_map_pca_projection,feats_pca_projection
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
import cv2
from imageio_ffmpeg import write_frames
from tqdm.notebook import trange, tqdm
from apex import amp
%load_ext autoreload
%autoreload 2
torch.cuda.set_device(2)

In [ ]:
train_folder = '/data/NFS/wmy/training'
files = [ i    for i in os.listdir(train_folder) if os.path.isdir(os.path.join(train_folder,i))]

#files = ['data19']
for data in files:
    if data =='data19':
        continue
    model_path = os.path.join(train_folder,data)
    fns = [ int(i.split('_')[2].split('.')[0])    for i in os.listdir(model_path) if 'nr_model_' in i]
    if len(fns)==0:
        continue
    fns.sort()
    fns = fns[-1]
    print(data,fns)
    render(model_path,fns)

In [ ]:
def render(model_path, epoch):
    para_file = 'nr_model_%d.pth' % epoch


    cfg.merge_from_file(os.path.join(model_path,'configs.yml'))
    #cfg.INPUT.SIZE_TRAIN = [1000,750]
    #cfg.INPUT.SIZE_TEST = [1000,750]
    cfg.SOLVER.IMS_PER_BATCH = 1
    #cfg.freeze()
    
    test_loader, dataset = make_data_loader(cfg, is_train=True)
    
    model = build_model(cfg, isTrain = False)
    a = torch.load(os.path.join(model_path,para_file),map_location='cpu')
    model.pcpr_parameters.setPointNum(a['pcpr_parameters.p_parameters'].size(1))
    model.load_state_dict(a)
    #model.eval()
    model = model.cuda()
    #optimizer = make_optimizer(cfg, model)
    #model, optimizer = amp.initialize(model, optimizer, opt_level="O1")
    
    feature_maps = []
    tars = []


    i = 0
    for batch in test_loader:
        in_points = batch[1].cuda()
        K = batch[2].cuda()
        T = batch[3].cuda()
        near_far_max_splatting_size = batch[5]
        num_points = batch[4]
        point_indexes = batch[0]
        target = batch[7].cuda()
        inds = batch[6].cuda()
        rgbs = batch[8].cuda()

        res,depth,features,dir_in_world,rgb,m_point_features = model(in_points, K, T,
                            near_far_max_splatting_size, num_points, rgbs, inds)

        #fused_features = torch.cat([features,dir_in_world],dim = 1)
        #feature_maps.append(fused_features.detach().cpu().half())
        #tars.append((target.cpu()*255).byte())


        #if i %100 == 0:
        #    print(i,'/',len(test_loader))
        i = i+1

        if (i>0):
            break
            
    #plt.figure(figsize=(10,10))
    plt.imshow(res.detach().cpu()[0].permute(1,2,0)[:,:,0:3])
    plt.show()
    

    
    import numpy as np
    from scipy.spatial.transform import Rotation as R
    def slerp(p0, p1, t):
            omega = np.arccos(np.dot(p0/np.linalg.norm(p0), p1/np.linalg.norm(p1)))
            so = np.sin(omega)+1e-4
            return np.sin((1.0-t)*omega) / so * p0 + np.sin(t*omega)/so * p1

    def gen_trajectories(T,num=3.0):
        N = T.shape[0]
        new_T = []
        for i in range(1,N):
            r0 = R.from_matrix(T[i-1,0:3,0:3])
            r1 = R.from_matrix(T[i,0:3,0:3])

            for t in np.arange(0.0, 1.0, 1.0/num):
                n_t = np.zeros((4,4))
                n_t[0:3,0:3] = R.from_quat(slerp(r0.as_quat(), r1.as_quat(), t)).as_matrix()
                n_t[0:3,3] = (T[i,0:3,3] - T[i-1,0:3,3])*t + T[i-1,0:3,3]
                n_t[3,3] = 1.0
                new_T.append(n_t)
        new_T = np.stack(new_T, axis=0)
        return new_T

    new_T = gen_trajectories(dataset.datasets[0].Ts[0:-1:4,:,:].numpy(),num=8)

    index = 0

    dx = 1

    if not os.path.exists(os.path.join(model_path,'vis_%d'%epoch)):
        os.mkdir(os.path.join(model_path,'vis_%d'%epoch))
    #if not os.path.exists(os.path.join(model_path,'vis_mask_%d'%epoch)):
    #    os.mkdir(os.path.join(model_path,'vis_mask_%d'%epoch))
    #if not os.path.exists(os.path.join(model_path,'vis_depth_%d'%epoch)):
    #    os.mkdir(os.path.join(model_path,'vis_depth_%d'%epoch))

    #if not os.path.exists(os.path.join(model_path,'vis_compos_%d'%epoch)):
    #    os.mkdir(os.path.join(model_path,'vis_compos_%d'%epoch))

    sKs = []
    sTs = []
    frames_id = []


    writer_raw_rgb = write_frames(os.path.join(os.path.join(model_path,'vis_%d'%epoch),'vis_frame%04d_gt.mp4'%index), cfg.INPUT.SIZE_TRAIN,fps=30, macro_block_size =8,quality=6)  # size is (width, height)
    writer_raw_depth = write_frames(os.path.join(os.path.join(model_path,'vis_%d'%epoch),'vis_frame%04d_depth.mp4'%index), cfg.INPUT.SIZE_TRAIN,fps=30, macro_block_size =8,quality=6)  # size is (width, height)


    writer_raw_rgb.send(None)
    writer_raw_depth.send(None)


    for index in range(1):
        _,in_points, _,_,_,_,rgbs,_ =  dataset.__getitem__(index*dataset.datasets[0].Ts.size(0))


        print('frame:',index)



        for j in range(1):



            #s_pos = (center - radius*v.numpy()  + up*radius*0.5*j + up*radius*0.2)


            for i in tqdm(range(new_T.shape[0])):
            #for i in range(longitude_step(j)):


                #near_far_max_splatting_size = torch.tensor([[0.5000, 6.5000, 0.2]])

                num_points = torch.Tensor([in_points.size(0)])
                rgbs = rgbs.cuda()
                #print(num_points)

                in_points = in_points.cuda()



                nR = new_T[i]

                sTs.append(nR)
                sKs.append(K[0].cpu().numpy())
                frames_id.append('img_%04d_%d_%d.jpg'%(index,i,j))

                T[0,:,:] = torch.Tensor(nR).cuda()
                with torch.no_grad():
                    res,depth,features,dir_in_world,rgb,m_point_features = model(in_points, K, T,
                                        near_far_max_splatting_size, num_points, rgbs, inds)

                img_t = res.detach().cpu()[0]
                img_t[img_t>1.0] = 1.0
                img_t[img_t<0] = 0


                #mask_t[mask_t<0.95] = 0
                #mask_t[mask_t>0] = 1

                img_t[0:3,:,:] = img_t[0:3,:,:]
                img = cv2.cvtColor(img_t.permute(1,2,0).numpy()*255.0,cv2.COLOR_BGR2RGB)
                img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

                img_depth = depth.detach().cpu()[0][0].numpy()
                img_depth = img_depth *255/ np.max(img_depth)
                img_depth = cv2.cvtColor(img_depth,cv2.COLOR_GRAY2RGB)
                
                


                #pca_feature = feats_map_pca_projection(features[0].detach().cpu().numpy())
                #m_point_features = m_point_features.transpose(0,1).detach().cpu().numpy()

                #pca_p_feats =  feats_pca_projection(m_point_features)

                #writer.add_image('vis/rendered', pca_feature.transpose(2,0,1), i)
                #writer.add_mesh('vis/points', vertices = in_points.view(1,in_points.size(0),-1) ,colors = torch.Tensor(pca_p_feats).view(1,in_points.size(0),-1),global_step = i)

                #cv2.imwrite(os.path.join(model_path,'vis_%d/img_%04d_%d_%d.jpg'%(epoch,index,i,j)),img)

                if not os.path.exists(os.path.join(os.path.join(model_path,'vis_%d'%epoch), '%04d'% index)):
                    os.mkdir(os.path.join(os.path.join(model_path,'vis_%d'%epoch), '%04d'% index))
                #img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                cv2.imwrite(os.path.join(os.path.join(os.path.join(model_path,'vis_%d'%epoch), '%04d'% index),'img_frame%04d_%04d.bmp'%(index,i)),img.astype(np.uint8))
                


                writer_raw_rgb.send(img.astype(np.uint8))



                writer_raw_depth.send(img_depth.astype(np.uint8))




                del res
                del depth
                del features
                del dir_in_world
                del rgb
                del img
                torch.cuda.empty_cache()
                #print(i,'/360')

        with open(os.path.join(model_path,'Intrinsic_%d_gt.inf'%epoch), 'w') as f:
            for i,camk in enumerate(sKs):
                f.write('%d\n'%i)
                f.write('%f %f %f\n %f %f %f\n %f %f %f\n' % tuple(camk.reshape(9).tolist()))
                f.write('\n')


        with open(os.path.join(model_path,'CamPose_%d_gt.inf' %epoch), 'w') as f:
            for i,camT in enumerate(sTs):
                A = camT[0:3,:]
                tmp = np.concatenate( [A[0:3,2].T, A[0:3,0].T,A[0:3,1].T,A[0:3,3].T])
                f.write('%f %f %f %f %f %f %f %f %f %f %f %f\n' % tuple(tmp.tolist()))

        with open(os.path.join(model_path,'frames_%d_gt.inf' %epoch), 'w') as f:
            for i,ids in enumerate(frames_id):
                f.write('%s\n' % ids)


    writer_raw_rgb.close()
    writer_raw_depth.close()    

In [ ]:
reset

# Spiral

In [ ]:
def render(model_path, epoch):
    para_file = 'nr_model_%d.pth' % epoch


    cfg.merge_from_file(os.path.join(model_path,'configs.yml'))
    #cfg.INPUT.SIZE_TRAIN = [1000,750]
    #cfg.INPUT.SIZE_TEST = [1000,750]
    cfg.SOLVER.IMS_PER_BATCH = 1
    #cfg.freeze()
    
    test_loader, dataset = make_data_loader(cfg, is_train=False)
    
    model = build_model(cfg, isTrain = False)
    a = torch.load(os.path.join(model_path,para_file),map_location='cpu')
    model.pcpr_parameters.setPointNum(a['pcpr_parameters.p_parameters'].size(1))
    model.load_state_dict(a)
    #model.eval()
    model = model.cuda()
    #optimizer = make_optimizer(cfg, model)
    #model, optimizer = amp.initialize(model, optimizer, opt_level="O1")
    
    feature_maps = []
    tars = []


    i = 0
    for batch in test_loader:
        in_points = batch[1].cuda()
        K = batch[2].cuda()
        T = batch[3].cuda()
        near_far_max_splatting_size = batch[5]
        num_points = batch[4]
        point_indexes = batch[0]
        target = batch[7].cuda()
        inds = batch[6].cuda()
        rgbs = batch[8].cuda()

        res,depth,features,dir_in_world,rgb,m_point_features = model(in_points, K, T,
                            near_far_max_splatting_size, num_points, rgbs, inds)

        #fused_features = torch.cat([features,dir_in_world],dim = 1)
        #feature_maps.append(fused_features.detach().cpu().half())
        #tars.append((target.cpu()*255).byte())


        #if i %100 == 0:
        #    print(i,'/',len(test_loader))
        i = i+1

        if (i>0):
            break
            
    #plt.figure(figsize=(10,10))
    plt.imshow(res.detach().cpu()[0].permute(1,2,0)[:,:,0:3])
    plt.show()
    

    _,in_points, _,_,_,_,rgbs,_ =  dataset.__getitem__(0)
    center = torch.mean(in_points,dim=0).cpu()
    up = -torch.mean(dataset.datasets[0].Ts[:,0:3,1],dim =0)
    up = up / torch.norm(up)
    radius = torch.norm(dataset.datasets[0].Ts[0,0:3,3] - center) *0.87

    center = center + up*radius*0.1


    #v = torch.tensor([0,0,-1], dtype=torch.float32)
    v = dataset.datasets[0].Ts[0,0:3,3] - center
    v = v / torch.norm(v)


    s_pos = center + v * radius + up*radius*0.05

    center = center.numpy()
    up = up.numpy()
    radius = radius.item()
    s_pos = s_pos.numpy()

    lookat = center - s_pos
    lookat = lookat/np.linalg.norm(lookat)

    xaxis = np.cross(lookat, up)
    xaxis = xaxis / np.linalg.norm(xaxis)

    import math
    def rotate(angle):
        #res = np.array([ [math.cos(angle), -math.sin(angle), 0],[ math.sin(angle),math.cos(angle),0],[0,0,1] ])
        res = np.array([ [math.cos(angle), 0, math.sin(-angle)],[0,1,0],[ math.sin(-angle),0, math.cos(angle)]])
        return res

    def rodrigues_rotation_matrix(axis, theta):
        axis = np.asarray(axis)
        theta = np.asarray(theta)
        axis = axis/math.sqrt(np.dot(axis, axis))
        a = math.cos(theta/2.0)
        b, c, d = -axis*math.sin(theta/2.0)
        aa, bb, cc, dd = a*a, b*b, c*c, d*d
        bc, ad, ac, ab, bd, cd = b*c, a*d, a*c, a*b, b*d, c*d
        return np.array([[aa+bb-cc-dd, 2*(bc+ad), 2*(bd-ac)],
                         [2*(bc-ad), aa+cc-bb-dd, 2*(cd+ab)],
                         [2*(bd+ac), 2*(cd-ab), aa+dd-bb-cc]])

    index = 0

    dx = 1

    if not os.path.exists(os.path.join(model_path,'vis_%d'%epoch)):
        os.mkdir(os.path.join(model_path,'vis_%d'%epoch))
    #if not os.path.exists(os.path.join(model_path,'vis_mask_%d'%epoch)):
    #    os.mkdir(os.path.join(model_path,'vis_mask_%d'%epoch))
    #if not os.path.exists(os.path.join(model_path,'vis_depth_%d'%epoch)):
    #    os.mkdir(os.path.join(model_path,'vis_depth_%d'%epoch))

    #if not os.path.exists(os.path.join(model_path,'vis_compos_%d'%epoch)):
    #    os.mkdir(os.path.join(model_path,'vis_compos_%d'%epoch))

    sKs = []
    sTs = []
    frames_id = []


    latitde = 1

    longitude_step = lambda x: int((1.0 - float(abs(x))/latitde)*120 + 40)


    writer_raw_rgb = write_frames(os.path.join(os.path.join(model_path,'vis_%d'%epoch),'vis_frame%04d.mp4'%index), cfg.INPUT.SIZE_TRAIN,fps=30, macro_block_size =8,quality=6)  # size is (width, height)
    writer_raw_depth = write_frames(os.path.join(os.path.join(model_path,'vis_%d'%epoch),'vis_frame%04d_depth.mp4'%index), cfg.INPUT.SIZE_TRAIN,fps=30, macro_block_size =8,quality=6)  # size is (width, height)
    writer_raw_compose = write_frames(os.path.join(os.path.join(model_path,'vis_%d'%epoch),'vis_frame%04d_compose.mp4'%index), cfg.INPUT.SIZE_TRAIN,fps=30, macro_block_size =8,quality=6)  # size is (width, height)


    writer_raw_rgb.send(None)
    writer_raw_depth.send(None)
    writer_raw_compose.send(None)


    for index in range(1):
        _,in_points, _,_,_,_,rgbs,_ =  dataset.__getitem__(index*dataset.datasets[0].Ts.size(0))


        print('frame:',index)



        for j in range(1):



            #s_pos = (center - radius*v.numpy()  + up*radius*0.5*j + up*radius*0.2)


            for i in trange(longitude_step(j)):
            #for i in range(longitude_step(j)):


                #near_far_max_splatting_size = torch.tensor([[0.5000, 6.5000, 0.2]])

                num_points = torch.Tensor([in_points.size(0)])
                rgbs = rgbs.cuda()
                #print(num_points)

                in_points = in_points.cuda()


                angle_step = 180.0/ longitude_step(j)

                angle = 3.1415926*4*i*angle_step/360.0

                #angle= 0 
                s_pos = (center + radius*v.numpy()  + up*radius*0.0*math.sin(3.1415926*2*i/longitude_step(j)) + up*radius*0.05)
                pos = s_pos - center
                pos = rodrigues_rotation_matrix(up,-angle).dot(pos) 

                pos = pos + center

                #print('pos:',pos)


                lookat = center - pos
                lookat = lookat/np.linalg.norm(lookat)

                xaxis = np.cross(lookat, up)
                xaxis = xaxis / np.linalg.norm(xaxis)

                yaxis = -np.cross(xaxis,lookat)
                yaxis = yaxis/np.linalg.norm(yaxis)

                nR = np.array([xaxis,yaxis,lookat, pos]).T
                nR = np.concatenate([nR,np.array([[0,0,0,1]])])

                sTs.append(nR)
                sKs.append(K[0].cpu().numpy())
                frames_id.append('img_%04d_%d_%d.jpg'%(index,i,j))

                T[0,:,:] = torch.Tensor(nR).cuda()
                with torch.no_grad():
                    res,depth,features,dir_in_world,rgb,m_point_features = model(in_points, K, T,
                                        near_far_max_splatting_size, num_points, rgbs, inds)

                img_t = res.detach().cpu()[0]
                img_t[img_t>1.0] = 1.0
                img_t[img_t<0] = 0


                #mask_t[mask_t<0.95] = 0
                #mask_t[mask_t>0] = 1

                img_t[0:3,:,:] = img_t[0:3,:,:]
                img = cv2.cvtColor(img_t.permute(1,2,0).numpy()*255.0,cv2.COLOR_BGR2RGB)
                img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

                img_depth = depth.detach().cpu()[0][0].numpy()
                img_depth = img_depth *255/ np.max(img_depth)
                img_depth = cv2.cvtColor(img_depth,cv2.COLOR_GRAY2RGB)
                
                depth_mask = depth[:,0:1,:,:].detach().cpu().clone()
                depth_mask[depth_mask>0] = 1
                depth_mask = torch.clamp(depth_mask,0,1.0).repeat(1,3,1,1)

                #pca_feature = feats_map_pca_projection(features[0].detach().cpu().numpy())
                #m_point_features = m_point_features.transpose(0,1).detach().cpu().numpy()

                #pca_p_feats =  feats_pca_projection(m_point_features)

                #writer.add_image('vis/rendered', pca_feature.transpose(2,0,1), i)
                #writer.add_mesh('vis/points', vertices = in_points.view(1,in_points.size(0),-1) ,colors = torch.Tensor(pca_p_feats).view(1,in_points.size(0),-1),global_step = i)

                #cv2.imwrite(os.path.join(model_path,'vis_%d/img_%04d_%d_%d.jpg'%(epoch,index,i,j)),img)

                if not os.path.exists(os.path.join(os.path.join(model_path,'vis_%d'%epoch), '%04d'% index)):
                    os.mkdir(os.path.join(os.path.join(model_path,'vis_%d'%epoch), '%04d'% index))
                #img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                
               

                
                writer_raw_rgb.send(img.astype(np.uint8))
                
                #print((img_t*depth_mask).size())
                img_mask = (depth_mask)[0].permute(1,2,0)
                
                img_mask = cv2.cvtColor(img_mask.numpy()*255.0,cv2.COLOR_BGR2RGB)
                img_mask = cv2.cvtColor(img_mask,cv2.COLOR_BGR2RGB)
                
                img_mask = img_mask.astype(np.uint8)
                
                kernel = np.ones((5,5),np.uint8)
                closing = cv2.morphologyEx(img_mask, cv2.MORPH_CLOSE, kernel)
                
                closing[closing>240] = 255
                closing[closing<255] = 0
                
                cv2.imwrite(os.path.join(os.path.join(os.path.join(model_path,'vis_%d'%epoch), '%04d'% index),'img_frame%04d_%04d.jpg'%(index,i)),img.astype(np.uint8))
                cv2.imwrite(os.path.join(os.path.join(os.path.join(model_path,'vis_%d'%epoch), '%04d'% index),'img_frame%04d_%04d_mask.jpg'%(index,i)),closing)



                writer_raw_depth.send(closing.astype(np.uint8))
                
                img[closing<200] = 255
                writer_raw_compose.send(img.astype(np.uint8))




                del res
                del depth
                del features
                del dir_in_world
                del rgb
                del img
                torch.cuda.empty_cache()
                #print(i,'/360')

        with open(os.path.join(model_path,'Intrinsic_%d.inf'%epoch), 'w') as f:
            for i,camk in enumerate(sKs):
                f.write('%d\n'%i)
                f.write('%f %f %f\n %f %f %f\n %f %f %f\n' % tuple(camk.reshape(9).tolist()))
                f.write('\n')


        with open(os.path.join(model_path,'CamPose_%d.inf' %epoch), 'w') as f:
            for i,camT in enumerate(sTs):
                A = camT[0:3,:]
                tmp = np.concatenate( [A[0:3,2].T, A[0:3,0].T,A[0:3,1].T,A[0:3,3].T])
                f.write('%f %f %f %f %f %f %f %f %f %f %f %f\n' % tuple(tmp.tolist()))

        with open(os.path.join(model_path,'frames_%d.inf' %epoch), 'w') as f:
            for i,ids in enumerate(frames_id):
                f.write('%s\n' % ids)


    writer_raw_rgb.close()
    writer_raw_depth.close()    
    writer_raw_compose.close()   

In [ ]:
angle = 3.1415926*2*i/360.0

pos = s_pos - center
pos = rodrigues_rotation_matrix(up,-angle).dot(pos) 
pos = pos + center

print(pos)

In [ ]:

new_T

In [ ]:
feats = features.detach()[0].cpu().numpy().transpose(1,2,0)
print(feats.shape)
feats = np.reshape(feats,(-1,32))
print(feats.shape)

In [ ]:
data = np.mat(feats.T)
data_mean = data - data.mean(axis=1)
data_cov = np.cov(data_mean)
tzz,tzxl = np.linalg.eig(data_cov)

xl = tzxl.T[0:3]

res =  data_mean.T.__mul__(np.mat(xl).T)

res = res/np.std(res) + 0.5
res = np.array(res)
print(res.shape)

In [ ]:
res = res.reshape((600,800,3))

In [ ]:
plt.figure(figsize=(16,16))
plt.imshow(res)

In [ ]:
T1 =  np.vstack( [np.array([ -0.991663, -0.075813, 0.104196, 0.062654, -0.990274, -0.124229, 0.112601, -0.116665, 0.986768,-0.315234 ,-0.048224, -7.282628]).reshape(4,3).T, np.array([0,0,0,1])])
T2 = np.vstack( [np.array([0.291734, 0.239859, -0.925937, 0.109816, 0.953247, 0.281534, 0.950174 ,-0.183816, 0.251754 , -4.038317, -0.059477, 4.109609]).reshape(4,3).T, np.array([0,0,0,1])])
Ts = T2.dot(np.linalg.inv(T1))
print(Ts)

In [ ]:
print(dir(dataset),dataset.frame_num)

In [ ]:
import math
def rodrigues_rotation_matrix(axis, theta):
    axis = np.asarray(axis)
    theta = np.asarray(theta)
    axis = axis/math.sqrt(np.dot(axis, axis))
    a = math.cos(theta/2.0)
    b, c, d = -axis*math.sin(theta/2.0)
    aa, bb, cc, dd = a*a, b*b, c*c, d*d
    bc, ad, ac, ab, bd, cd = b*c, a*d, a*c, a*b, b*d, c*d
    return np.array([[aa+bb-cc-dd, 2*(bc+ad), 2*(bd-ac)],
                     [2*(bc-ad), aa+cc-bb-dd, 2*(cd+ab)],
                     [2*(bd+ac), 2*(cd-ab), aa+dd-bb-cc]])

In [ ]:
#print(rodrigues_rotation_matrix([0,1,1],0.5))
import numpy as np
print(np.rad2deg(3.141592653))

In [ ]:
img_depth = depth.detach().cpu()[0][0].numpy()
img_depth = img *255/ np.max(img_depth)

In [ ]:
with open(os.path.join(model_path,'Intrinsic_%d.inf'%epoch), 'w') as f:
    for i,camk in enumerate(sKs):
        f.write('%d\n'%i)
        f.write('%f %f %f\n %f %f %f\n %f %f %f\n' % tuple(camk.reshape(9).tolist()))
        f.write('\n')
        
        
with open(os.path.join(model_path,'CamPose_%d.inf' %epoch), 'w') as f:
    for i,camT in enumerate(sTs):
        A = camT[0:3,:]
        tmp = np.concatenate( [A[0:3,2].T, A[0:3,0].T,A[0:3,1].T,A[0:3,3].T])
        f.write('%f %f %f %f %f %f %f %f %f %f %f %f\n' % tuple(tmp.tolist()))
        

        
with open(os.path.join(model_path,'frames_%d.inf' %epoch), 'w') as f:
    for i,ids in enumerate(frames_id):
        f.write('%d\n' % int(ids))

In [ ]:
frames_id =  np.array(frames_id).astype(np.int)

In [ ]:
print(T[0,0:3,0:3])

In [ ]:
from scipy.spatial.transform import Rotation as R

In [ ]:
r = R.from_matrix([[0, -1, 0],
               [1, 0, 0],
                [0, 0, 1]])

In [ ]:
r = scipy.spatial.transform.Rotation([[0, -1, 0],
               [1, 0, 0],
                [0, 0, 1]])


In [ ]:
import scipy

In [ ]:
r.as_euler('xyz')

In [ ]:
import numpy as np
import scipy
from scipy.spatial.transform import Rotation as R
def slerp(p0, p1, t):
        omega = np.arccos(np.dot(p0/np.linalg.norm(p0), p1/np.linalg.norm(p1)))
        so = np.sin(omega)
        return np.sin((1.0-t)*omega) / so * p0 + np.sin(t*omega)/so * p1

def gen_trajectories(T,num=3.0):
    N = T.shape[0]
    new_T = []
    for i in range(1,N):
        key_rots = R.from_matrix(T[i-1:i+1,0:3,0:3])
        key_times = [0, 1]
        slerp = scipy.spatial.transform.Slerp(key_times, key_rots)
        times = np.arange(0.0, 1.0, 1.0/num)
        interp_rots = slerp(times)
        
        Rs = [i for i in interp_rots.as_matrix()]
        
        for t in np.arange(0.0, 1.0, 1.0/num):
            n_t = np.zeros((4,4))
            n_t[0:3,0:3] = i
            n_t[0:3,3] = (T[i,0:3,3] - T[i-1,0:3,3])*t + T[i-1,0:3,3]
            n_t[3,3] = 1.0
            new_T.append(n_t)
    new_T = np.stack(new_T, axis=0)
    return new_T
    
    

In [ ]:
gen_trajectories(dataset.datasets[0].Ts[0:2,:,:].numpy(),num=3)

In [ ]:
p = R.random(5, random_state=2342345)

In [ ]:
for i in p:
    print(i)